In [5]:
#loading dependencies
import numpy as np
import math
from pprint import pprint
import random
import operator
%matplotlib inline
import matplotlib as mpl

import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import matplotlib #only needed to determine Matplotlib version number

from os import listdir
from os.path import isfile, join
import csv

In [6]:
def load_data(path):   
    # ==> Loading Merged Station Data
    data = pd.read_json(path)
    # ==> Rebuilding Multiindex
    data.set_index(["number",'last_update'],inplace=True,drop=True)
    # ==> Sort index by number & timestamp
    data.sort_index(inplace=True)
    return data

In [7]:
data = load_data('challenge_data/stations_merged.json')

In [8]:
data.columns.values

array([u'address', u'available_bike_stands', u'available_bikes',
       u'banking', u'bike_stands', u'bonus', u'contract_name', u'index',
       u'lat', u'lng', u'name', u'status'], dtype=object)

In [9]:
data.head()


address  available_bike_stands  \
number last_update                                                          
1010   1441065103000  10 RUE BOUCHER - 75001 PARIS                     25   
       1441065707000  10 RUE BOUCHER - 75001 PARIS                     25   
       1441066310000  10 RUE BOUCHER - 75001 PARIS                     25   
       1441066913000  10 RUE BOUCHER - 75001 PARIS                     25   
       1441067517000  10 RUE BOUCHER - 75001 PARIS                     25   

                      available_bikes banking  bike_stands  bonus  \
number last_update                                                  
1010   1441065103000                0    True           25  False   
       1441065707000                0    True           25  False   
       1441066310000                0    True           25  False   
       1441066913000                0    True           25  False   
       1441067517000                0    True           25  False   

                     contract_name  index        lat       lng  \
number last_update                                               
1010   1441065103000         Paris  14015  48.859462  2.344366   
       1441065707000         Paris   7503  48.859462  2.344366   
       1441066310000         Paris  12609  48.859462  2.344366   
       1441066913000         Paris    557  48.859462  2.344366   
       1441067517000         Paris  10262  48.859462  2.344366   

                                   name status  
number last_update                              
1010   1441065103000  01010 - PONT NEUF   OPEN  
       1441065707000  01010 - PONT NEUF   OPEN  
       1441066310000  01010 - PONT NEUF   OPEN  
       1441066913000  01010 - PONT NEUF   OPEN  
       1441067517000  01010 - PONT NEUF   OPEN

In [67]:
data['available_bikes'].value_counts(dropna = False)

address                  0
available_bike_stands    0
available_bikes          0
banking                  0
bike_stands              0
bonus                    0
contract_name            0
index                    0
lat                      0
lng                      0
name                     0
status                   0
dtype: int64

In [11]:
# Method that checks the actual capacity (available bike stands + available bikes)
def capacity_delta(row):
    return row['bike_stands'] - (row['available_bike_stands'] + row['available_bikes'])

# Apply method to dataset
capacity_delta = data.apply(capacity_delta, axis = 1)

KeyboardInterrupt: 

In [15]:

capacity_delta.counts()

AttributeError: 'function' object has no attribute 'counts'

In [95]:
# Create the delta (mean = 0) of theoretical capacity vs actual capacity
def capacity_delta(row):
    return row['bike_stands'] - row['actual_capacity']
    
    
data['capacity_delta'] = data.apply(capacity_delta, axis = 1)

In [96]:

data.head()

address  available_bike_stands  \
number last_update                                                          
1010   1441065103000  10 RUE BOUCHER - 75001 PARIS                     25   
       1441065707000  10 RUE BOUCHER - 75001 PARIS                     25   
       1441066310000  10 RUE BOUCHER - 75001 PARIS                     25   
       1441066913000  10 RUE BOUCHER - 75001 PARIS                     25   
       1441067517000  10 RUE BOUCHER - 75001 PARIS                     25   

                      available_bikes banking  bike_stands  bonus  \
number last_update                                                  
1010   1441065103000                0    True           25  False   
       1441065707000                0    True           25  False   
       1441066310000                0    True           25  False   
       1441066913000                0    True           25  False   
       1441067517000                0    True           25  False   

                     contract_name  index        lat       lng  \
number last_update                                               
1010   1441065103000         Paris  14015  48.859462  2.344366   
       1441065707000         Paris   7503  48.859462  2.344366   
       1441066310000         Paris  12609  48.859462  2.344366   
       1441066913000         Paris    557  48.859462  2.344366   
       1441067517000         Paris  10262  48.859462  2.344366   

                                   name status  actual_capacity  \
number last_update                                                
1010   1441065103000  01010 - PONT NEUF   OPEN               25   
       1441065707000  01010 - PONT NEUF   OPEN               25   
       1441066310000  01010 - PONT NEUF   OPEN               25   
       1441066913000  01010 - PONT NEUF   OPEN               25   
       1441067517000  01010 - PONT NEUF   OPEN               25   

                      capacity_delta  
number last_update                    
1010   1441065103000               0  
       1441065707000               0  
       1441066310000               0  
       1441066913000               0  
       1441067517000               0

In [139]:
from __future__ import division
pivot = data['capacity_delta'].value_counts(dropna = False).sort_values()

ratio = pivot.map(lambda x: x/tot)
print pd.concat([pivot, ratio], axis=1)


     capacity_delta  capacity_delta
 31               1        0.000001
 22               1        0.000001
 26               1        0.000001
 24               1        0.000001
-8                2        0.000003
 30               2        0.000003
 21               3        0.000004
 23               6        0.000008
 28               8        0.000010
-3                9        0.000012
 34              10        0.000013
 19              15        0.000019
-2               16        0.000021
 18              24        0.000031
-1               57        0.000074
 27             108        0.000140
 33             145        0.000188
 17             177        0.000229
 15             181        0.000234
 14             751        0.000972
 11             768        0.000994
 13             835        0.001081
 12             911        0.001179
 16            1515        0.001961
 10            1673        0.002166
 9             4220        0.005463
 8             5875        0

In [110]:
pivot_capacity = data['capacity_delta'].value_counts(dropna = False)
type(pivot_capacity)

pandas.core.series.Series

In [62]:
# average available_bike_stands and available_bikes 
data.pivot_table( index = 'address', columns ='status', aggfunc = [np.mean], values = ['available_bike_stands', 'available_bikes'])

mean  \
                                                available_bike_stands   
status                                                         CLOSED   
address                                                                 
1 AVENUE DE LA MOTTE PICQUET - 75007 PARIS                  18.000000   
1 QUAI AUX FLEURS - 75004 PARIS                              2.666667   
10 RUE BOUCHER - 75001 PARIS                                19.000000   
10 RUE D'ARCOLE - 75004 PARIS                                6.000000   
105-109 TERRE PLEIN SAINT PAUL - 75004 PARIS                16.625000   
13 RUE SURCOUF - 75007 PARIS                                      NaN   
165 RUE SAINT HONORE - 75001 PARIS                          14.000000   
17 RUE DUROC - 75007 PARIS                                        NaN   
186 RUE SAINT HONORE - 75001 PARIS                          43.200000   
2 AVENUE OCTAVE CREARD - 75007 PARIS                         4.000000   
2 PLACE ANDRE MALRAUX - 75001 PARIS                          7.500000   
2 QUAI DES CELESTINS - 75004 PARIS                           6.200000   
2 RUE CAMBON - 75001 PARIS                                   0.000000   
2 RUE D'ALGER - 75001 PARIS                                  9.400000   
2 RUE DE L'ORATOIRE - 75001 PARIS                           10.666667   
2 RUE TIRON - 75004 PARIS                                   22.333333   
20 RUE COQUILLIERE - 75001 PARIS                            12.125000   
215 RUE SAINT HONORE - 75001 PARIS                           7.000000   
23 AVENUE DE SEGUR - 75007 PARIS                             8.750000   
237 RUE SAINT HONORE - 75001 PARIS                           5.222222   
27 RUE THERESE - 75001 PARIS                                 4.142857   
29 RUE DES BLANCS MANTEAUX - 75004 PARIS                    21.000000   
3 AVENUE BOSQUET - 75007 PARIS                                    NaN   
30 BIS RUE LAS CASES - 75007 PARIS                           5.666667   
35 BOULEVARD DES INVALIDES - 75007 PARIS                    23.571429   
37 AVENUE BOSQUET - 75007 PARIS                                   NaN   
37 RUE CASANOVA - 75001 PARIS                                2.714286   
39 QUAI D'ORSAY - 75007 PARIS                                9.166667   
43 AVENUE RAPP - 75007 PARIS                                 4.000000   
49 RUE BERGER - 75001 PARIS                                 17.333333   
5 RUE DE L'ECHELLE - 75001 PARIS                            23.250000   
50 RUE VIEILLE DU TEMPLE - 75004 PARIS                       9.307692   
6 RUE SAINT PAUL - 75004 PARIS                               6.250000   
7 CITE VANEAU - 75007 PARIS                                 33.000000   
85 AVENUE BOSQUET - 75007 PARIS                                   NaN   
9 BOULEVARD DES INVALIDES - 75007 PARIS                           NaN   
BOULEVARD BOURDON - 75004 PARIS                             21.777778   
FACE 1 BOULEVARD BOURBON - 75004 PARIS                      41.500000   
FACE 18 RUE DE L'HOTEL DE VILLE - 75004 PARIS               12.142857   
FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS             15.000000   
FACE 3 RUE DE CONSTANTINE - 75007 PARIS                      1.714286   
FACE 3 RUE DU CHAMP DE MARS - 75007 PARIS                   28.000000   
PLACE DU LIEUTENANT HENRI KARCHER - 75001 PARIS             24.250000   
PLACE LOUIS LEPINE - 75004 PARIS                            11.000000   
QUAI BRANLY - 75007 PARIS                                         NaN   

                                                                            \
                                                           available_bikes   
status                                                OPEN          CLOSED   
address                                                                      
1 AVENUE DE LA MOTTE PICQUET - 75007 PARIS       10.109542       20.000000   
1 QUAI AUX FLEURS - 75004 PARIS                   9.526560       17.333333   
10 RUE BOUCHER - 75

In [64]:
# min/max available_bikes 
data.pivot_table( index = 'address', columns ='status', aggfunc = [np.min, np.max], values = ['available_bikes'])

amin       \
                                                available_bikes        
status                                                   CLOSED OPEN   
address                                                                
1 AVENUE DE LA MOTTE PICQUET - 75007 PARIS                 20.0  0.0   
1 QUAI AUX FLEURS - 75004 PARIS                            14.0  0.0   
10 RUE BOUCHER - 75001 PARIS                                4.0  0.0   
10 RUE D'ARCOLE - 75004 PARIS                              25.0  0.0   
105-109 TERRE PLEIN SAINT PAUL - 75004 PARIS                6.0  0.0   
13 RUE SURCOUF - 75007 PARIS                                NaN  0.0   
165 RUE SAINT HONORE - 75001 PARIS                          5.0  0.0   
17 RUE DUROC - 75007 PARIS                                  NaN  0.0   
186 RUE SAINT HONORE - 75001 PARIS                         15.0  0.0   
2 AVENUE OCTAVE CREARD - 75007 PARIS                       27.0  0.0   
2 PLACE ANDRE MALRAUX - 75001 PARIS                         8.0  0.0   
2 QUAI DES CELESTINS - 75004 PARIS                          6.0  0.0   
2 RUE CAMBON - 75001 PARIS                                 40.0  0.0   
2 RUE D'ALGER - 75001 PARIS                                 6.0  0.0   
2 RUE DE L'ORATOIRE - 75001 PARIS                           7.0  0.0   
2 RUE TIRON - 75004 PARIS                                   4.0  0.0   
20 RUE COQUILLIERE - 75001 PARIS                            3.0  0.0   
215 RUE SAINT HONORE - 75001 PARIS                         12.0  0.0   
23 AVENUE DE SEGUR - 75007 PARIS                            5.0  0.0   
237 RUE SAINT HONORE - 75001 PARIS                          6.0  0.0   
27 RUE THERESE - 75001 PARIS                               15.0  0.0   
29 RUE DES BLANCS MANTEAUX - 75004 PARIS                    8.0  0.0   
3 AVENUE BOSQUET - 75007 PARIS                              NaN  0.0   
30 BIS RUE LAS CASES - 75007 PARIS                         13.0  0.0   
35 BOULEVARD DES INVALIDES - 75007 PARIS                   18.0  0.0   
37 AVENUE BOSQUET - 75007 PARIS                             NaN  0.0   
37 RUE CASANOVA - 75001 PARIS                              18.0  0.0   
39 QUAI D'ORSAY - 75007 PARIS                              11.0  0.0   
43 AVENUE RAPP - 75007 PARIS                               25.0  0.0   
49 RUE BERGER - 75001 PARIS                                 3.0  0.0   
5 RUE DE L'ECHELLE - 75001 PARIS                            0.0  0.0   
50 RUE VIEILLE DU TEMPLE - 75004 PARIS                      0.0  0.0   
6 RUE SAINT PAUL - 75004 PARIS                             20.0  0.0   
7 CITE VANEAU - 75007 PARIS                                 3.0  0.0   
85 AVENUE BOSQUET - 75007 PARIS                             NaN  0.0   
9 BOULEVARD DES INVALIDES - 75007 PARIS                     NaN  0.0   
BOULEVARD BOURDON - 75004 PARIS                            10.0  0.0   
FACE 1 BOULEVARD BOURBON - 75004 PARIS                     10.0  0.0   
FACE 18 RUE DE L'HOTEL DE VILLE - 75004 PARIS              19.0  0.0   
FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS             7.0  0.0   
FACE 3 RUE DE CONSTANTINE - 75007 PARIS                    64.0  0.0   
FACE 3 RUE DU CHAMP DE MARS - 75007 PARIS                  28.0  1.0   
PLACE DU LIEUTENANT HENRI KARCHER - 75001 PARIS             6.0  0.0   
PLACE LOUIS LEPINE - 75004 PARIS                           11.0  0.0   
QUAI BRANLY - 75007 PARIS                                   NaN  1.0   

                                                           amax        
                                                available_bikes        
status                                                   CLOSED  OPEN  
address                                                                
1 AVENUE DE LA MOTTE PICQUET - 75007 PARIS                 20.0  39.0  
1 QUAI AUX FLEURS - 75004 PARIS                            20.0  21.0  
10 RUE BOUCHER - 75001 PARIS                                8.0  25.0  
10 RUE D'ARCOLE - 75004 PA

In [94]:
data['bike_stands'].value_counts(dropna = False)

25    62804
20    51372
36    50731
21    49466
22    47093
47    36161
14    34963
28    34064
31    33353
18    32428
26    20332
27    19383
69    19112
30    18943
32    18787
66    18618
50    18259
65    18251
58    17978
60    17948
42    17760
40    17738
55    17396
57    17226
33    17093
53    16669
39    16643
24    16032
37    15914
Name: bike_stands, dtype: int64

In [147]:
# number of timestamps on which 
data.groupby(['address', 'status']).head()

address  \
number last_update                                                      
1010   1441065103000                     10 RUE BOUCHER - 75001 PARIS   
       1441065707000                     10 RUE BOUCHER - 75001 PARIS   
       1441066310000                     10 RUE BOUCHER - 75001 PARIS   
       1441066913000                     10 RUE BOUCHER - 75001 PARIS   
       1441067517000                     10 RUE BOUCHER - 75001 PARIS   
       1444630053000                     10 RUE BOUCHER - 75001 PARIS   
       1444630420000                     10 RUE BOUCHER - 75001 PARIS   
       1444630715000                     10 RUE BOUCHER - 75001 PARIS   
       1444630992000                     10 RUE BOUCHER - 75001 PARIS   
1012   1441065179000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1441065782000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1441066385000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1441066989000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1441067593000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1444983570000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1444983811000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1444984130000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1444984414000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
       1444984539000  FACE 29 RUE JEAN JACQUES ROUSSEAU - 75001 PARIS   
1013   1441064943000               186 RUE SAINT HONORE - 75001 PARIS   
       1441065546000               186 RUE SAINT HONORE - 75001 PARIS   
       1441066150000               186 RUE SAINT HONORE - 75001 PARIS   
       1441066753000               186 RUE SAINT HONORE - 75001 PARIS   
       1441067357000               186 RUE SAINT HONORE - 75001 PARIS   
       1445240682000               186 RUE SAINT HONORE - 75001 PARIS   
       1445240943000               186 RUE SAINT HONORE - 75001 PARIS   
       1445241269000               186 RUE SAINT HONORE - 75001 PARIS   
       1445241475000               186 RUE SAINT HONORE - 75001 PARIS   
       1445241824000               186 RUE SAINT HONORE - 75001 PARIS   
1014   1441065084000                 5 RUE DE L'ECHELLE - 75001 PARIS   
...                                                               ...   
7023   1441066279000                        QUAI BRANLY - 75007 PARIS   
       1441066883000                        QUAI BRANLY - 75007 PARIS   
       1441067306000                        QUAI BRANLY - 75007 PARIS   
7024   1441065237000                     43 AVENUE RAPP - 75007 PARIS   
       1441065841000                     43 AVENUE RAPP - 75007 PARIS   
       1441066444000                     43 AVENUE RAPP - 75007 PARIS   
       1441066895000                     43 AVENUE RAPP - 75007 PARIS   
       1441067499000                     43 AVENUE RAPP - 75007 PARIS   
       1445601524000                     43 AVENUE RAPP - 75007 PARIS   
       1445601773000                     43 AVENUE RAPP - 75007 PARIS   
       1445602178000                     43 AVENUE RAPP - 75007 PARIS   
       1445602482000                     43 AVENUE RAPP - 75007 PARIS   
7025   1441065370000             2 AVENUE OCTAVE CREARD - 75007 PARIS   
       1441065973000             2 AVENUE OCTAVE CREARD - 75007 PARIS   
       1441066577000             2 AVENUE OCTAVE CREARD - 75007 PARIS   
       1441067318000             2 AVENUE OCTAVE CREARD - 75007 PARIS   
       1441067922000             2 AVENUE OCTAVE CREARD - 75007 PARIS   
       1445600329000             2 AVENUE OCTAVE CREARD - 75007 PARIS   
       1445600718000             2 AVENUE OCTAVE CREARD - 75007 PARIS   
       1445601322000             2 AVENUE OCTAVE CREARD - 75007 PARIS   
7026   1441065532000                    39 QUAI D'ORSAY - 75007 PARIS   
       1441066136000                    39 QUAI D'ORSAY - 75007 PARIS   
       1441066740000                    39 QUAI